# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [2]:
import pandas as pd

## imports for Python, Pandas

In [3]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,state,shortname
0,Dade,12345,Rick Scott,Florida,FL
1,Broward,40000,Rick Scott,Florida,FL
2,Palm Beach,60000,Rick Scott,Florida,FL
3,Summit,1234,John Kasich,Ohio,OH
4,Cuyahoga,1337,John Kasich,Ohio,OH


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [4]:
# load json as string
sample = json.load((open('data/world_bank_projects_less.json')))

In [40]:
#json.load((open('data/world_bank_projects_less.json')))

In [5]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{u'$oid': u'52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{u'Percent': 100, u'Name': u'Education for all'}","[{u'code': u'65', u'name': u'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{u'$oid': u'52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{u'Percent': 30, u'Name': u'Other economic man...","[{u'code': u'24', u'name': u'Other economic ma...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [6]:
json_df = pd.read_json('data/world_bank_projects.json')

### 1. 10 countries with most projects

Making sure that normalization is not needed at this point (one project per 'level')

In [7]:
json_df[['countrycode','countryshortname','project_name']].head(10)

,countrycode,countryshortname,project_name
0,ET,Ethiopia,Ethiopia General Education Quality Improvement...
1,TN,Tunisia,TN: DTF Social Protection Reforms Support
2,TV,Tuvalu,Tuvalu Aviation Investment Project - Additiona...
3,RY,"Yemen, Republic of",Gov't and Civil Society Organization Partnership
4,LS,Lesotho,Second Private Sector Competitiveness and Econ...
5,KE,Kenya,Additional Financing for Cash Transfers for Or...
6,IN,India,National Highways Interconnectivity Improvemen...
7,CN,China,China Renewable Energy Scale-Up Program Phase II
8,IN,India,Rajasthan Road Sector Modernization Project
9,MA,Morocco,MA Accountability and Transparency DPL


Okay. Good, so this is a series with top 10 countries with most projects

In [8]:
top_10 = json_df['countrycode'].value_counts().head(10)

In [9]:
top_10

CN    19
ID    19
VN    17
IN    16
RY    13
BD    12
MA    12
NP    12
3A    11
MZ    11
Name: countrycode, dtype: int64

### 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

Use normalization because there are multiple project themes. List everything in a flat table and then do value counts.

In [42]:
#json_normalize(sample, 'mjtheme_namecode', ['countrycode','countryshortname','project_name'])

In [69]:
# load json as string
data = json.load((open('data/world_bank_projects.json')))

#### Normalize

In [75]:
df = json_normalize(data, 'mjtheme_namecode', ['countrycode','countryshortname','project_name'])

In [76]:
df.code.value_counts()

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
3      15
Name: code, dtype: int64

#### Looking for missing name
### 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [77]:
df[df['code'] == '11'].head()

,code,name,project_name,countryshortname,countrycode
1,11,,Ethiopia General Education Quality Improvement...,Ethiopia,ET
6,11,Environment and natural resources management,Tuvalu Aviation Investment Project - Additiona...,Tuvalu,TV
16,11,Environment and natural resources management,China Renewable Energy Scale-Up Program Phase II,China,CN
28,11,Environment and natural resources management,Uttarakhand Disaster Recovery Project,India,IN
31,11,Environment and natural resources management,Timor Leste Road Climate Resilience Project - ...,Timor-Leste,TP


In [78]:
#

Okay. So the names are missing, let us fill the 'missing' data.

In [79]:
df['code'] = df['code'].astype('category') #changed datatype

In [80]:
counter = 0
for code in list(df['code'].cat.categories):
    """ This will loop through a list of all the codes and find corresponding names that should exit in name column.
        Then populate the name column with the appropriate name"""
    #print (df['name'][df['code'] == code].value_counts().index.tolist()) #to verify that no other alternate names exist
    name = df['name'][df['code'] == code].value_counts().index.tolist()[0]
    df['name'].loc[df['code'] == code] = name
    counter += 1
    
    #print (df['name'][df['code'] == code].head())

print counter

11


In [81]:
df[df['code'] == '11'].head() #sample recheck

,code,name,project_name,countryshortname,countrycode
1,11,Environment and natural resources management,Ethiopia General Education Quality Improvement...,Ethiopia,ET
6,11,Environment and natural resources management,Tuvalu Aviation Investment Project - Additiona...,Tuvalu,TV
16,11,Environment and natural resources management,China Renewable Energy Scale-Up Program Phase II,China,CN
28,11,Environment and natural resources management,Uttarakhand Disaster Recovery Project,India,IN
31,11,Environment and natural resources management,Timor Leste Road Climate Resilience Project - ...,Timor-Leste,TP


### Now that we have completed 3., let's continue with 2.

In [82]:
df.head()

,code,name,project_name,countryshortname,countrycode
0,8,Human development,Ethiopia General Education Quality Improvement...,Ethiopia,ET
1,11,Environment and natural resources management,Ethiopia General Education Quality Improvement...,Ethiopia,ET
2,1,Economic management,TN: DTF Social Protection Reforms Support,Tunisia,TN
3,6,Social protection and risk management,TN: DTF Social Protection Reforms Support,Tunisia,TN
4,5,Trade and integration,Tuvalu Aviation Investment Project - Additiona...,Tuvalu,TV


In [83]:
df['name'].value_counts().head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64